In [ ]:
# %env CUBLAS_WORKSPACE_CONFIG=:4096:8

In [ ]:
import random
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn.init as init
from google.colab import files
from io import BytesIO

random.seed(3242023)
torch.manual_seed(3242023) # Seeded with a constant, so that behavior is deterministic.
torch.cuda.manual_seed_all(3242023)

# torch.use_deterministic_algorithms(True)

if torch.cuda.is_available():
    device = "cuda"
else:
   device = "cpu"

PRETRAIN_FROM_SCRATCH = True # If True, pretrains a Resnet20. If False, uploads a pretrained Resnet20
samples = 5000
sample_mode = "independent" # Should be one of "independent" or an integer number of samples to exclude from each training
pretraining_epochs = 1000
pretraining_samples = 50000
pretraining_batch_size = 2000 # Set equal to pretraining_samples if you don't want to use batches.
epochs = 250
num_trials = 25

print(f"Pytorch running on {device}")

In [ ]:
def _weights_init(m):
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                """
                For CIFAR10 ResNet paper uses option A.
                """
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def resnet20():
    return ResNet(BasicBlock, [3, 3, 3]).to(device)

"""
All code above here with the exception of setting the random seed was taken from
https://github.com/akamaster/pytorch_resnet_cifar10/blob/master/resnet.py
"""

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x

def get_params(model):
    params = []
    for param in model.parameters():
        params.append(param.view(-1))
    params = torch.cat(params)
    return params

In [ ]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

def test_model(model, preprocessing = lambda x: x):
    test_loss = 0.0
    testloader = torch.utils.data.DataLoader(testset, shuffle=True, num_workers=2, batch_size = 20)
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    train_on_gpu = torch.cuda.is_available()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    model.eval()
    # iterate over test data
    for data, target in testloader:
        # move tensors to GPU if CUDA is available
        data, target = data.to(device), target.to(device)
        # preprocess data with an arbitrary function
        data = preprocessing(data)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update test loss
        test_loss += loss.item()*data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)
        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
        for i in range(20):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
    # average test loss
    test_loss = test_loss/len(testloader.dataset)
    print('Test Loss: {:.6f}\n'.format(test_loss))

    for i in range(10):
        if class_total[i] > 0:

            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
              classes[i], 100 * class_correct[i] / class_total[i],
              np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))
    print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))
    return 100. * np.sum(class_correct) / np.sum(class_total)

In [ ]:
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD_DEV = (0.2023, 0.1994, 0.2010)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD_DEV),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

net = resnet20()
net.to(device)

if not PRETRAIN_FROM_SCRATCH:
    uploaded = files.upload()
    torch_object = torch.load(BytesIO(list(uploaded.values())[0]), map_location=device)
    net.load_state_dict(torch_object)
else:
    pretrain_subset = torch.utils.data.Subset(trainset, random.sample(range(50000), pretraining_samples))

trainloader = torch.utils.data.DataLoader(pretrain_subset, batch_size=pretraining_batch_size,
                                        shuffle=False)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=False)

params = []

In [ ]:
if PRETRAIN_FROM_SCRATCH:
    print("Beginning pretraining...")
    net.train()
    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
    for epoch in range(pretraining_epochs):
        for data in iter(trainloader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch} of pretraining successfully completed")
    torch.save(net.state_dict(), "PretrainedResnet20.pt")
    files.download("PretrainedResnet20.pt")

In [ ]:
print("Test loss after pretraining:")
test_model(net)
net.linear = Identity()
net.eval()
preprocessing = net

In [ ]:
class DenseModel(nn.Module): # TODO replace with the last layer of pretrained

    def __init__(self):
        super(DenseModel, self).__init__()
        self.layer1 = nn.Linear(64, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 32)
        self.layer4 = nn.Linear(32, 16)
        self.layer5 = nn.Linear(16, 16)
        self.layer6 = nn.Linear(16, 10)

    def forward(self, x):
        out = F.relu(self.layer1(x))
        out = F.relu(self.layer2(out))
        out = F.relu(self.layer3(out))
        out = F.relu(self.layer4(out))
        out = F.relu(self.layer5(out))
        out = self.layer6(out)
        return out

In [ ]:
net_base = DenseModel()
net_base.to(device)
params = []
print("Beginning training...")
for trial in range(num_trials):
    net = DenseModel()
    net.to(device)
    net.load_state_dict(net_base.state_dict()) # Ensure we start from the same place with both models. This appears to be the crucial step.

    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
    if sample_mode == "independent":
        train_subset = torch.utils.data.Subset(trainset, random.sample(range(50000), samples))
    else:
        train_subset = torch.utils.data.Subset(trainset, random.sample(range(samples), samples-sample_mode))
    trainloader = torch.utils.data.DataLoader(train_subset, batch_size=samples,
                                          shuffle=False)
    net.train()

    print(f"Beginning trial #{trial}")
    for epoch in range(epochs):
        for data in iter(trainloader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs = preprocessing(inputs)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch} of trial {trial} successfully completed with {loss} train loss")
    params.append(get_params(net))
    print(f"Test loss at the conclusion of trial {trial}:")
    test_model(net, preprocessing)


In [ ]:
params = torch.stack(params)
torch.save(params, f"DenseWithPretrainedtrials{num_trials}X.pt")
files.download(f"DenseWithPretrainedtrials{num_trials}X.pt")